In [ ]:
%load_ext autoreload
%autoreload 2

In [4]:
from obs.collector import Online_Collector
from obs.query import RT_Visualization
from azureml.core import Workspace
import pandas as pd

ws = Workspace.from_config()


Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azure-identity 1.9.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azure-identity==1.7.0'), {'azureml-dataprep'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core.

### Real time metrics logging and visualization
#### You need to run this using your local computer or if you run with Azure ML Compute Instance, please do so from VSCode as Dash visualization requires access to localhost

In [9]:
# Ingest streaming data  asynchronously with internal buffering mechanism to lower impact to main scoring thread
streaming_table_name="streaming_test2"
streaming_collector = Online_Collector(streaming_table_name,ws=ws)

import random
streaming_collector.start_logging_daemon(buffer_time=2, batch_size=10)

for run_id in ["r000001", "r000002", "r000003", "r000004", "r000005"]:
    for i in range(200):
        for lr in ["0.001", "0.002"]:
            df = pd.DataFrame({ "timestamp":pd.to_datetime('now'), "lr":[lr],"metric1":[random.uniform(3,50)] })
            streaming_collector.stream_collect_df_queue(df)
# streaming_collector.stop_logging_daemon()

.create table streaming_test2 (['timestamp']: datetime, ['lr']: string, ['metric1']: real)


In [16]:
from plotly.subplots import make_subplots
rt_viz =RT_Visualization(streaming_table_name,ws)
rt_viz.scatter(max_records=200, ago='15m',groupby='lr', y_metric='metric1',x_metric='timestamp')

### Model Monitoring

The deploy_model utility install AML CLI v2, create managed online endpoint, deploy the model. These steps are optional incase you want to supply your own names and objects. Check the function to see all paramters. The below just create everything.

In [1]:
from deployment.deploy_util import deploy_model,score_test

scoring_uri, scoring_key = deploy_model(ws=ws)

Once you have the model deployed with the collector object inside, you can just call it to simulate data creation. Data is collected to ADX table nIRIS_MODEL.

In [18]:
import time
#If you run this without running deployment, the look up scoring_uri and scoring key from Azure ML Studio
scoring_uri= "https://iris-ep.westus2.inference.ml.azure.com/score"
scoring_key="uPboOpl7zoiN5FVT2au3neWjnZCrp8oO"
for _ in range(200):
    time.sleep(3)
    score_test(scoring_uri, scoring_key)

b'["virginica", "setosa", "setosa", "virginica", "setosa", "virginica", "virginica", "virginica", "virginica", "setosa"]'
b'["setosa", "virginica", "setosa", "setosa", "setosa", "virginica", "setosa", "versicolor", "setosa", "setosa"]'
b'["virginica", "setosa", "setosa", "setosa", "virginica", "versicolor", "versicolor", "virginica", "versicolor", "virginica"]'
b'["virginica", "virginica", "versicolor", "virginica", "versicolor", "setosa", "setosa", "versicolor", "virginica", "virginica"]'
b'["virginica", "virginica", "setosa", "versicolor", "versicolor", "virginica", "versicolor", "virginica", "versicolor", "versicolor"]'
b'["versicolor", "virginica", "setosa", "virginica", "setosa", "setosa", "versicolor", "setosa", "versicolor", "versicolor"]'
b'["virginica", "virginica", "setosa", "setosa", "versicolor", "versicolor", "setosa", "setosa", "virginica", "virginica"]'
b'["setosa", "versicolor", "setosa", "versicolor", "versicolor", "virginica", "versicolor", "setosa", "versicolor", "ve

KeyboardInterrupt: 

---------------------------------------------------------------------------
ConnectionResetError                      Traceback (most recent call last)
c:\Users\janguy\Anaconda3\envs\dlresearch\lib\site-packages\urllib3\connectionpool.py in urlopen(
    self=<urllib3.connectionpool.HTTPSConnectionPool object>,
    method='POST',
    url='/v2/rest/query',
    body=b'{"db": "db01", "csl": "streaming_test2|where t...equesttimeout\\": true}, \\"Parameters\\": {}}"}',
    headers={'User-Agent': 'python-requests/2.26.0', 'Accept...aEIDpNjRRmOcYHZy2qxEmg', 'Content-Length': '176'},
    retries=Retry(total=0, connect=None, read=False, redirect=None, status=None),
    redirect=False,
    assert_same_host=False,
    timeout=Timeout(connect=3630, read=3630, total=None),
    pool_timeout=None,
    release_conn=False,
    chunked=False,
    body_pos=None,
    **response_kw={'decode_content': False, 'preload_content': False}
)
    669             # Make the request on the httplib connection object.


### Monitoring Dashboard


Review https://docs.microsoft.com/en-us/azure/data-explorer/azure-data-explorer-dashboards#to-create-new-dashboard-from-a-file for instruction on how to import the dashboard.


Go to ADX Dashboard and import the quick_start/ADX_dashboards/databoard_Model_Monitoring.json to create a monitoring dashboard for your model.


In [ ]:
# from obs import KV_ADX_URI, KV_ADX_DB
# for dashboard_file in ["ADX dashboards/dashboard-drift_detection.json", "ADX dashboards/dashboard-Model_Monitoring.json"]:
#     with open(dashboard_file, "r+") as temp_file:
#         dashboard_content = temp_file.readlines()
#         dashboard_content= dashboard_content[0]
#         kv = ws.get_default_keyvault()
#         cluster_uri = kv.get_secret(KV_ADX_URI)
#         cluster_db = kv.get_secret(KV_ADX_DB)
#         dashboard_content.replace("db01", cluster_db)
#         dashboard_content.replace("https://adx02.westus2.kusto.windows.net", cluster_uri)
#         temp_file.write(dashboard_content)